In [2]:
%pip install pydantic_settings langchain langchain-core langchain-google-genai langchain-qdrant fastembed langchain-community qdrant-client langgraph


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    GOOGLE_API_KEY: str
    model_config = SettingsConfigDict(env_file=".env")

env = Settings()

In [4]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings

# embeddings_2 = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=env.GOOGLE_API_KEY)


# Optionally, you can use FastEmbed for embeddings
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
embeddings_2 = FastEmbedEmbeddings(cache_dir="./embedding_cache", model_name="jinaai/jina-embeddings-v2-base-en")
# # https://qdrant.github.io/fastembed/examples/Supported_Models/#supported-text-embedding-models

In [5]:
from qdrant_client.http.models import Distance

collection_name = "mcu_packages"
dimension = 768
distance = Distance.COSINE

# Create Vector Data

In [6]:
# load mcu.json data
import json

with open("doctors_final.json", "r") as f:
    mcu_data = json.load(f)

print(mcu_data[0])

{'id': '5eae0017-40dd-4961-869f-79d9e45d87f2', 'name': 'Adventia Emilia Krysna Sipi Seda, M.M., M.Psi., Psikolog', 'specialization_name': 'Psikologi', 'specialization_name_en': 'Psychology', 'sub_specialization_name': 'Psikolog', 'sub_specialization_name_en': 'Psychologist', 'hospital_name': 'Siloam Hospitals Yogyakarta'}


In [7]:
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")

In [8]:
from qdrant_client.http.models import VectorParams

if(client.collection_exists(collection_name=collection_name) == False):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=dimension, distance=distance),
    )

In [9]:
from qdrant_client.models import PointStruct
import uuid
i = 0
for row in mcu_data:
    i += 1
    text = f"Doctor Name: {row['name']}, spesializ: {row['specialization_name']}, spesializ en: {row['specialization_name_en']}, sub spesializ: {row['sub_specialization_name']}, sub spesializ en: {row['sub_specialization_name_en']}, hospital_name: {row['hospital_name']}"
    emb = embeddings_2.embed_query(text)
    print(i)
    client.upsert(
        collection_name=collection_name,
        points=[
            PointStruct(
                id=str(uuid.uuid4()),  # Generate a unique ID for each point
                vector=emb, 
                payload={
                    "page_content": text,
                    "metadata": {
                            "id": row['id'],
                            "name": row['name'],
                            "specialization_name": row['specialization_name'],
                            "specialization_name_en": row['specialization_name_en'],
                            "sub_specialization_name": row['sub_specialization_name'],
                            "sub_specialization_name_en": row['sub_specialization_name_en'],
                            "hospital_name": row['hospital_name'],

                    },
                },
            )
        ],
    )
    print(text)

1
Doctor Name: Adventia Emilia Krysna Sipi Seda, M.M., M.Psi., Psikolog, spesializ: Psikologi, spesializ en: Psychology, sub spesializ: Psikolog, sub spesializ en: Psychologist, hospital_name: Siloam Hospitals Yogyakarta
2
Doctor Name: Dokter Umum Siloam Yogyakarta, spesializ: Kedokteran Umum, spesializ en: General Practitioner, sub spesializ: Dokter Umum, sub spesializ en: General Practitioner, hospital_name: Siloam Hospitals Yogyakarta
3
Doctor Name: dr. Andrean Jefrian Manihuruk, spesializ: Kedokteran Umum, spesializ en: General Practitioner, sub spesializ: Dokter Umum, sub spesializ en: General Practitioner, hospital_name: Siloam Hospitals Yogyakarta
4
Doctor Name: dr. Anggoro Budi Hartopo, M.Sc., Ph.D., SpPD., SpJP (K), spesializ: Kardiologi (Jantung), spesializ en: Cardiology (Heart), sub spesializ: Subspesialis Prevensi dan Rehabilitasi Kardiovaskular, sub spesializ en: Preventive Cardiology and Cardiac Rehabilitation Subspecialist, hospital_name: Siloam Hospitals Yogyakarta
5
D

In [10]:
from langchain_qdrant import QdrantVectorStore
def get_retriever():

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embeddings_2,
    )
    
    return vector_store.as_retriever()

In [11]:
from langchain_core.tools import tool
from typing import Annotated, List

@tool
def search_mcu_packages(query: Annotated[str, "search query must contain keywords related to MCU packages"]) -> List[str]:
    """Search for MCU packages by name or description."""
    retriever = get_retriever()
    results = retriever.invoke(query, k=10)
    return [result.page_content for result in results]

In [12]:
search_mcu_packages("Orthopaedics, Hip and Knee Subspecialist")

/var/folders/s2/d0szrj810bx2dsxzm9m4hgnw0000gn/T/ipykernel_4352/3367345619.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search_mcu_packages("Orthopaedics, Hip and Knee Subspecialist")


['Doctor Name: dr. Ardi Setiawan, SpOT (K) Hip & Knee, FIPP, spesializ: Ortopedi (Tulang), spesializ en: Orthopaedics (Bone), sub spesializ: Subspesialis Panggul dan Lutut, sub spesializ en: Hip and Knee Subspecialist, hospital_name: Siloam Hospitals TB Simatupang',
 'Doctor Name: Dr. dr. Franky Hartono, SpOT (K), spesializ: Ortopedi (Tulang), spesializ en: Orthopaedics (Bone), sub spesializ: Subspesialis Panggul dan Lutut, sub spesializ en: Hip and Knee Subspecialist, hospital_name: Siloam Hospitals Kebon Jeruk',
 'Doctor Name: dr. Dadang Rona Sasetyo, SpOT (K), spesializ: Ortopedi (Tulang), spesializ en: Orthopaedics (Bone), sub spesializ: Subspesialis Panggul dan Lutut, sub spesializ en: Hip and Knee Subspecialist, hospital_name: Siloam Hospitals ASRI',
 'Doctor Name: dr. Erwin Saspraditya, B.Med Sci, SpOT (K), spesializ: Ortopedi (Tulang), spesializ en: Orthopaedics (Bone), sub spesializ: Subspesialis Panggul dan Lutut, sub spesializ en: Hip and Knee Subspecialist, hospital_name: S

# Create Agent

In [13]:
# access the Google Gemini API
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=env.GOOGLE_API_KEY,
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that provides information about Siloam hospitals."),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

In [14]:
chain.invoke({"question": "Saya mau bertemu dokter spesialis Orthopaedics, Hip and Knee Subspecialist di Siloam Hospital. Siapa saja yang bisa saya temui?"})

AIMessage(content='Berikut adalah daftar dokter spesialis Orthopaedics, Hip and Knee Subspecialist yang dapat Anda temui di Siloam Hospitals:\n\n*   **dr. Achmad Fauzi, Sp.OT (K)** - Siloam Hospitals Kebon Jeruk\n*   **dr. Franky Hartono, Sp.OT (K)** - Siloam Hospitals Lippo Village\n*   **dr. IGM Febry Siswanto, Sp.OT (K)** - Siloam Hospitals Denpasar\n*   **dr. Ketut Ngurah Gunapriya, Sp.OT (K)** - Siloam Hospitals Denpasar\n*   **dr. Luthfi Gatam, Sp.OT (K)** - Siloam Hospitals Mampang\n*   **dr. Muhammad Arsyad Subu, Sp.OT (K)** - Siloam Hospitals Makassar\n*   **dr. Yoga Yuniadi, Sp.OT (K)** - Siloam Hospitals Lippo Village\n\nUntuk membuat janji temu dengan dokter-dokter tersebut, Anda dapat mengunjungi situs web resmi Siloam Hospitals atau menghubungi pusat panggilan Siloam Hospitals di 1-500-181.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []

# Workflow for agent to use tool

In [15]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
# Define state for application
class State(TypedDict):
    question: str
    context: List[str]
    search: str
    answer: str

In [16]:
def get_context(state: State):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """
              You are an expert in medical specialist matching.
            You will provide a keyword based on the question to help identify the appropriate medical specialist from a given list.
The keyword should be relevant to the medical condition, symptom, or concern mentioned in the question.
Do not provide any other information.
If the question already contains a clear keyword, return it as is.
Only return one keyword and in English.
            """),
            ("human", "{question}"),
        ]
    )
    chain = prompt | llm
    result = chain.invoke({"question": state["question"]})
    return {"search": result.content}

In [17]:
def retrieve(state: State):
    retrieved_docs = search_mcu_packages(state["search"])
    return {"context": retrieved_docs}

In [18]:
def generate(state: State):
    prompt = ChatPromptTemplate.from_messages(
        [
     
("system", """
    You are an assistant that helps users find the right doctors based on their medical concerns.
    You will generate a list of relevant doctors by matching the user's question to the appropriate specialty.
    Use the provided list of doctors with their specialties and hospital locations to find matches.
    
    doctor list knowledge:
    {context}
    
    If the context is empty, return a general message suggesting to consult a general practitioner.
    
    Always return:
    - the matched specialty
    - a list of relevant doctor names
    - the hospital name where each doctor practices

    Keep the response clear and concise.
"""),
("human", "{question}"),


        ]
    )
    chain = prompt | llm
    result = chain.invoke({"question": state["question"], "context": state["context"]})
    return {"answer": result.content}

In [19]:
graph_builder = StateGraph(State).add_sequence([get_context, retrieve, generate])
graph_builder.add_edge(START, "get_context")
graph = graph_builder.compile()

In [20]:
response = graph.invoke({
	"question": "saya ingin bertemu dokter gigi di jogja",
	"context": [],
	"search": "",
	"answer": ""
})
print(response["answer"])

- Specialty: Kedokteran Gigi (Dentistry)
- Doctor Name: drg. Bintang Charisma Putri Lolobua, MDSc
- Hospital Name: Siloam Hospitals Yogyakarta
